In [1]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-groq \
  langchain-community==0.3.16 \
  langsmith==0.3.4 \
  google-search-results==2.4.2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.3/333.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.7 requires langchain-core>=1.0.0, but you have langchain-core 0.3.33 which is incompatible.


In [2]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "langchain-agents"

Enter LangSmith API Key: ··········


# Introduction to Tools

In [3]:
from langchain_core.tools import tool

@tool
def add(x: float, y:float) -> float:
  """Add 'x' and 'y'."""
  return x + y

@tool
def multiply(x: float, y:float) -> float:
  """Multiply 'x' and 'y'."""
  return x * y

@tool
def exponentiate(x: float, y:float) -> float:
  """Raise 'x' to the power of 'y'."""
  return x ** y

@tool
def substract(x: float, y:float) -> float:
  """Substract 'x' from 'y'."""
  return y - x

With the @tool decorator our function is turned into a StructuredTool object, which we can see below:

In [4]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x79ee94e80540>)

In [5]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"        # output from the LLM
llm_output_dict = json.loads(llm_output_string)   # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

In [6]:
exponentiate.func(**llm_output_dict)

25

This is then passed into the tool function as kwargs (keyword arguments) as indicated by the ** operator - the ** operator is used to unpack the dictionary into keyword arguments.

# Creating an Agent

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [8]:
import os
from getpass import getpass
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY") or \
    getpass("Enter your GROQ API key: ")

llm = ChatGroq(temperature=0.7, model="llama-3.3-70b-versatile")

Enter your GROQ API key: ··········


When creating an agent we need to add conversational memory to make the agent remember previous interactions. We'll be using the older **ConversationBufferMemory** class rather than the newer **RunnableWithMessageHistory** — the reason being that we will also be using the older **create_tool_calling_agent** and **AgentExecutor** method and class.

In [9]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",        # must align with MessagesPlaceholder variable_name
    return_messages=True
)

/tmp/ipython-input-1076208803.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Now we will initialize our agent. For that we need:

* llm: as already defined
* tools: to be defined (just a list of our previously defined tools)
* prompt: as already defined
* memory: as already defined

In [10]:
from langchain.agents import create_tool_calling_agent

tools = [add, multiply, exponentiate, substract]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

In [11]:
memory.chat_memory.messages

[]

In [12]:
agent.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps": []   # agent will append its internal steps here
})

[ToolAgentAction(tool='multiply', tool_input={'x': 10.7, 'y': 7.68}, log="\nInvoking: `multiply` with `{'x': 10.7, 'y': 7.68}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '75rrfp463', 'function': {'arguments': '{"x":10.7,"y":7.68}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 437, 'total_tokens': 460, 'completion_time': 0.065427275, 'completion_tokens_details': None, 'prompt_time': 0.12613622, 'prompt_tokens_details': None, 'queue_time': 0.009263997, 'total_time': 0.191563495}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_43d97c5965', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0c1de128-047a-4586-ad41-1208a222aa49-0', tool_calls=[{'name': 'multiply', 'args': {'x': 10.7, 'y': 7.68}, 'id': '75rrfp463', 'type': 'tool_call'}], usage_metadata={'input_tokens': 437, 'output_tokens': 23, 'total_tokens': 460})], tool_call_id='75rrfp463')]

In [13]:
memory.chat_memory.messages

[]

In [14]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

In [15]:
memory.chat_memory.messages

[]

In [16]:
agent_executor.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 10.7, 'y': 7.68}`


82.17599999999999The result of 10.7 multiplied by 7.68 is 82.176.

> Finished chain.


{'input': 'what is 10.7 multiplied by 7.68?',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of 10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={})],
 'output': 'The result of 10.7 multiplied by 7.68 is 82.176.'}

'82.17599999999999' is tool output and 'The result of 10.7 multiplied by 7.68 is 82.176.' is LLM output


In [17]:
memory.chat_memory.messages

[HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The result of 10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={})]

In [18]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}), AIMessage(content='The result of 10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={})]), return_messages=True, memory_key='chat_history')

In [19]:
agent_executor.invoke({
    "input": "My name is puLeeNa",
    "chat_history": memory.chat_memory.messages
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 10.7, 'y': 7.68}`


82.17599999999999The result of 10.7 multiplied by 7.68 is 82.176.

> Finished chain.


{'input': 'My name is puLeeNa',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of 10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is puLeeNa', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of 10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={})],
 'output': 'The result of 10.7 multiplied by 7.68 is 82.176.'}

In [20]:
agent_executor.invoke({
    "input": "What is nine plus 10, minus 4 * 2, to the power of 3",
    "chat_history": memory
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 4, 'y': 2}`


8.0
Invoking: `substract` with `{'x': 4, 'y': 19}`


15.0
Invoking: `exponentiate` with `{'x': 15, 'y': 3}`


3375.0First, 4 * 2 equals 8. Then, 10 + 9 equals 19, and 19 - 8 equals 11. Lastly, 11 to the power of 3 equals 1331. The answer is 1331.

> Finished chain.


{'input': 'What is nine plus 10, minus 4 * 2, to the power of 3',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of 10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is puLeeNa', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of 10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='First, 4 * 2 equals 8. Then, 10 + 9 equals 19, and 19 - 8 equals 11. Lastly, 11 to the power of 3 equals 1331. The answer is 1331.', additional_kwargs={}, response_metadata={})],
 'output': 'First, 4 * 2 equals 8. Then, 10 + 9 equals 19, and 19 - 8 equals 11. Lastly, 11 to the power of 3 equals 1331. The answer is 1331.'}

In [21]:
agent_executor.invoke({
    "input": "What is my name",
    "chat_history": memory
})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'x': 9, 'y': 10}`


19.0
Invoking: `multiply` with `{'x': 4, 'y': 2}`


8.0
Invoking: `substract` with `{'x': 8, 'y': 19}`


11.0
Invoking: `exponentiate` with `{'x': 11, 'y': 3}`


1331.0Your name is PuLeeNa.

> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of 10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is puLeeNa', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of 10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='First, 4 * 2 equals 8. Then, 10 + 9 equals 19, and 19 - 8 equals 11. Lastly, 11 to the power of 3 equals 1331. The answer is 1331.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is PuLeeNa.', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is P

# SerpAPI Weather Agent

In [22]:
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY") \
    or getpass("Enter your SerpAPI API key: ")

Enter your SerpAPI API key: ··········


In [23]:
from langchain.agents import load_tools

toolbox = load_tools(tool_names=['serpapi'], llm=llm)

In [24]:
import requests
from datetime import datetime

@tool
def get_location_from_ip():
    """Get the geographical location based on the IP address."""
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        if 'loc' in data:
            latitude, longitude = data['loc'].split(',')
            data = (
                f"Latitude: {latitude},\n"
                f"Longitude: {longitude},\n"
                f"City: {data.get('city', 'N/A')},\n"
                f"Country: {data.get('country', 'N/A')}"
            )
            return data
        else:
            return "Location could not be determined."
    except Exception as e:
        return f"Error occurred: {e}"

@tool
def get_current_datetime() -> str:
    """Return the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


In [25]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [26]:
tools = toolbox + [get_current_datetime, get_location_from_ip]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

In [29]:
out = agent_executor.invoke({
    "input": (
        "I have a few questions, what is the date and time right now? and How is the weather where I am? Please give me degrees in Celsius"
    )
})



> Entering new AgentExecutor chain...

Invoking: `get_current_datetime` with `{}`


2026-02-02 14:39:23
Invoking: `get_location_from_ip` with `{}`


Latitude: 41.2619,
Longitude: -95.8608,
City: Council Bluffs,
Country: US
Invoking: `Search` with `weather in Council Bluffs, US in Celsius`


{'type': 'weather_result', 'temperature': '-2', 'unit': 'Celsius', 'precipitation': '3%', 'humidity': '94%', 'wind': '2 km/h', 'location': 'Council Bluffs, IA', 'date': 'Monday 8:00 AM', 'weather': 'Partly sunny'}The current date and time is February 2, 2026, 2:39 PM. The weather in Council Bluffs, US is partly sunny with a temperature of -2 degrees Celsius, 3% precipitation, 94% humidity, and a wind speed of 2 km/h.

> Finished chain.


In [30]:
from IPython.display import display, Markdown

display(Markdown(out["output"]))

The current date and time is February 2, 2026, 2:39 PM. The weather in Council Bluffs, US is partly sunny with a temperature of -2 degrees Celsius, 3% precipitation, 94% humidity, and a wind speed of 2 km/h.